In [439]:
import numpy as np
import pandas as pd
import lightgbm as lgb

!pip install optuna
import optuna
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

In [440]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score # f1_score, recall_score
import lightgbm as lgb

**1. 학습 데이터 생성**

In [441]:
# 데이터셋 가져오기

data = datasets.fetch_covtype()
X = data.data
y = data.target

In [442]:
# 랜덤한 1500 개의 데이터를 학습

num_samples = 1500
np.random.seed(2025)
indices = np.random.choice(X.shape[0], num_samples, replace=False)

X = X[indices]
y = y[indices]

In [443]:
# 데이터 확인하기

X = pd.DataFrame(X, columns=data.feature_names)

In [444]:
X

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39
0,3197.0,229.0,15.0,547.0,88.0,450.0,196.0,253.0,191.0,2960.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2483.0,21.0,8.0,218.0,40.0,150.0,215.0,223.0,145.0,1771.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2856.0,63.0,7.0,150.0,8.0,1271.0,227.0,225.0,132.0,2311.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3129.0,100.0,19.0,339.0,68.0,1693.0,248.0,211.0,84.0,2755.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2607.0,69.0,15.0,90.0,-18.0,1020.0,234.0,210.0,103.0,765.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,2625.0,341.0,16.0,350.0,19.0,2741.0,184.0,214.0,168.0,2635.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1496,2441.0,10.0,14.0,255.0,50.0,499.0,203.0,210.0,142.0,1194.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1497,2709.0,103.0,6.0,124.0,18.0,1641.0,231.0,232.0,134.0,3463.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1498,2300.0,288.0,12.0,85.0,20.0,1448.0,187.0,239.0,192.0,721.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [445]:
y = pd.DataFrame(y, columns=['target'])

In [446]:
y

,target
0,2
1,3
2,2
3,2
4,2
...,...
1495,2
1496,6
1497,2
1498,6


In [447]:
# Class 분포 확인

y.value_counts()

,count
target,
2,716
1,547
3,82
7,62
6,55
5,26
4,12


In [448]:
# train, test_hpo, test_new 데이터셋으로 Split
# overfitting 여부를 판단하려면 y의 class 분포를 일치시키는 것이 비교적 좋음

from sklearn.model_selection import train_test_split

# [ Entire Dataset ] ---> [ Train ] / [ Test_hpo ] / [ Test_new ]
X_train, X_test_new, y_train, y_test_new =\
    train_test_split(X, y,
                     test_size=0.2,
                     stratify=y,  # overfitting 여부 판단에 도움
                     random_state=2027)

X_train_, X_test_hpo, y_train_, y_test_hpo =\
    train_test_split(X_train, y_train,
                     test_size=0.2,
                     stratify=y_train,  # overfitting 여부 판단에 도움
                     random_state=2027)

In [449]:
# train data 확인

X_train_

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39
131,2646.0,139.0,16.0,283.0,6.0,875.0,242.0,233.0,115.0,598.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
562,2602.0,0.0,6.0,306.0,84.0,2407.0,212.0,229.0,155.0,488.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
696,3371.0,279.0,10.0,421.0,31.0,3142.0,192.0,242.0,189.0,1832.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
782,3028.0,98.0,8.0,85.0,10.0,1805.0,234.0,228.0,126.0,1683.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87,2419.0,80.0,15.0,309.0,83.0,598.0,239.0,211.0,98.0,1364.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1348,2945.0,71.0,9.0,90.0,16.0,4918.0,231.0,222.0,124.0,6234.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1049,3029.0,350.0,7.0,67.0,3.0,4662.0,208.0,229.0,159.0,842.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
545,2815.0,210.0,16.0,30.0,-4.0,1140.0,206.0,253.0,179.0,1749.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
540,3365.0,67.0,17.0,258.0,43.0,2058.0,235.0,205.0,97.0,3387.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [450]:
y_train_

,target
131,3
562,3
696,1
782,1
87,6
...,...
1348,2
1049,2
545,2
540,7


In [451]:
y_train_count = y_train_.value_counts()
y_train_normalized = y_train_.value_counts(normalize=True)

y_train_distrib = pd.DataFrame({'count': y_train_count, 'percentage': 100 * y_train_normalized})
y_train_distrib

,count,percentage
target,,
2,458,47.708333
1,350,36.458333
3,53,5.520833
7,40,4.166667
6,35,3.645833
5,17,1.770833
4,7,0.729167


In [452]:
# test_hpo 데이터 확인 (HPO 성능 최적화를 위한 metric 값 도출용)

X_test_hpo

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39
733,2662.0,99.0,19.0,162.0,49.0,2318.0,247.0,211.0,85.0,2432.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
847,3005.0,307.0,5.0,85.0,6.0,4230.0,207.0,237.0,169.0,636.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
945,3023.0,103.0,19.0,630.0,216.0,641.0,248.0,212.0,84.0,977.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
518,3134.0,336.0,9.0,240.0,-12.0,633.0,199.0,227.0,167.0,1273.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1364,3251.0,126.0,17.0,395.0,-70.0,488.0,246.0,226.0,102.0,4062.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676,2940.0,111.0,4.0,127.0,8.0,3362.0,227.0,235.0,143.0,2546.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
970,3301.0,135.0,5.0,469.0,121.0,5272.0,229.0,238.0,143.0,1140.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
960,2926.0,255.0,5.0,391.0,23.0,5504.0,207.0,243.0,174.0,3856.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1387,2592.0,90.0,14.0,218.0,-16.0,342.0,241.0,216.0,101.0,1224.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [453]:
y_test_hpo

,target
733,3
847,2
945,2
518,1
1364,2
...,...
676,2
970,2
960,2
1387,2


In [454]:
y_test_hpo_count = y_test_hpo.value_counts()
y_test_hpo_normalized = y_test_hpo.value_counts(normalize=True)

y_test_hpo_distrib = pd.DataFrame({'count': y_test_hpo_count, 'percentage': 100 * y_test_hpo_normalized})
y_test_hpo_distrib

,count,percentage
target,,
2,115,47.916667
1,87,36.250000
3,13,5.416667
7,10,4.166667
6,9,3.750000
5,4,1.666667
4,2,0.833333


In [455]:
# test_new 데이터 확인 (HPO 완료한 후 최적의 모델을 테스트할 새로운 데이터)

X_test_new

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_30,Soil_Type_31,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39
211,3339.0,102.0,16.0,391.0,6.0,976.0,245.0,217.0,95.0,3630.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
531,2653.0,186.0,2.0,190.0,1.0,819.0,219.0,240.0,157.0,1682.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1198,2961.0,175.0,22.0,324.0,30.0,1717.0,226.0,245.0,140.0,1129.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1076,2847.0,350.0,22.0,175.0,67.0,3091.0,176.0,198.0,157.0,484.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1309,3034.0,40.0,4.0,541.0,50.0,4310.0,220.0,231.0,147.0,1500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,3022.0,340.0,15.0,127.0,30.0,150.0,188.0,217.0,168.0,1365.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
504,3071.0,336.0,35.0,335.0,183.0,395.0,123.0,170.0,173.0,2040.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
349,3140.0,214.0,12.0,242.0,-2.0,2642.0,209.0,251.0,177.0,671.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
718,3259.0,9.0,6.0,309.0,25.0,218.0,214.0,228.0,152.0,853.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [456]:
y_test_new

,target
211,1
531,2
1198,2
1076,2
1309,2
...,...
459,2
504,1
349,2
718,1


In [457]:
# y_test_new 의 class 분포가 y_test_old 와 최대한 비슷해야 overfitting 여부 판별에 비교적 도움이 됨

y_test_new_count = y_test_new.value_counts()
y_test_new_normalized = y_test_new.value_counts(normalize=True)

y_test_new_distrib = pd.DataFrame({'count': y_test_new_count, 'percentage': 100 * y_test_new_normalized})
y_test_new_distrib

,count,percentage
target,,
2,143,47.666667
1,110,36.666667
3,16,5.333333
7,12,4.000000
6,11,3.666667
5,5,1.666667
4,3,1.000000


In [458]:
# 각 data의 corr-coef 분석

import plotly.figure_factory as ff

# Show corr-coef chart for given dataset
#
# args
# - X         : data to visualize
# - data_name : name for corr-coef chart title

def visualize_corr(X, data_name):
    corr = X.corr()

    fig = ff.create_annotated_heatmap(
                z=corr.round(decimals=2).to_numpy(),
                x=corr.columns.tolist(),
                y=corr.index.tolist(),
                xgap=1,
                ygap=1,
                zmax=1,
                zmin=-1,
                showscale=True,
                colorscale=['#F53', '#BBC', '#57F'],
                hoverongaps=True)

    fig.update_layout(title=f'correlation of {data_name}',
                      width=1500,
                      height=1100)

    fig.show()

In [459]:
# train data 분포 확인 (생략)

# XY_train_ = pd.concat([X_train_, y_train_], axis=1)
# visualize_corr(XY_train_, data_name='train data')

In [460]:
# HPO test data 분포 확인 (생략)

# XY_test_hpo = pd.concat([X_test_hpo, y_test_hpo], axis=1)
# visualize_corr(XY_test_hpo, data_name='test data (for HPO)')

In [461]:
# NEW test data 분포 확인 (생략)

# XY_test_new = pd.concat([X_test_new, y_test_new], axis=1)
# visualize_corr(XY_test_new, data_name='test data (NEW test data)')

In [462]:
# 향후 정확도 측정용

from sklearn.metrics import accuracy_score

**2. HyperOpt 를 이용한 성능 테스트**

In [463]:
# Domain Space 정의

domain_space = {
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(1.0)),
    'max_depth': hp.quniform('max_depth', 1, 100, q=1),
    'num_leaves': hp.quniform('num_leaves', 50, 4000, q=1),
    'min_data_in_leaf': hp.quniform('min_data_in_leaf', 1, 100, q=1),
    'lambda_l1': hp.uniform('lambda_l1', 0.0, 1.0),
    'lambda_l2': hp.uniform('lambda_l2', 0.0, 5.0)
}

In [464]:
# Objective Function 정의

models = []

def objective_hyperopt(params):
    for param_name in ['max_depth', 'num_leaves', 'min_data_in_leaf']:
        params[param_name] = int(params[param_name])

    model = lgb.LGBMClassifier(
        objective='multiclass',
        metric='multi_logloss',
        boosting_type='gbdt',
        num_iterations=10,  # 일반적으로 너무 작은 값이나, 여기서는 빠른 실험을 위해 사용
        verbosity=-1,
        **params
    )

    # LigthGBM 용 valid set 인 X_valid_lgbm, y_valid_lgbm 로 데이터를 분리
    X_train__, X_valid_lgbm, y_train__, y_valid_lgbm =\
        train_test_split(X_train_, y_train_,
                         test_size=0.25,
                         random_state=2025)

    # X_valid_lgbm, y_valid_lgbm 를 이용하여 LightGBM 모델을 valid
    model.fit(X_train__, y_train__,
              eval_set=[(X_valid_lgbm, y_valid_lgbm)])

    # HPO 용 test set인 X_test_hpo, y_test_hpo 를 이용하여 하이퍼파라미터 조합 성능 테스트
    preds_hpo = model.predict(X_test_hpo)

    accuracy_hpo = accuracy_score(preds_hpo, y_test_hpo)
#    f1_hpo = f1_score(preds_hpo, y_test_hpo)
#    recall_hpo = recall_score(preds_hpo, y_test_hpo)

    acc = f'{(100 * accuracy_hpo):6.2f}'
#    f1 = f'{(100 * f1_hpo):6.2f}'
#    reca = f'{(100 * recall_hpo):6.2f}'

    print(f'[HPO] metrics with {params} : acc={acc}%') #, f1={f1}%, recall={reca}%')

    # 학습한 모델을 최종 테스트를 위해 배열에 저장
    models.append(model)

    # Accuracy 를 최대화 하도록 Loss 반환
    return {"loss": (-1) * accuracy_hpo, "status": STATUS_OK}

In [465]:
# 디버깅 중 불필요한 warning 메시지 미표시

import warnings
warnings.filterwarnings("ignore")

In [466]:
# 하이퍼파라미터 최적화 실시

TRIAL_COUNT = 3000

trials = Trials()
best_params_hyperopt = fmin(fn=objective_hyperopt,
                            space=domain_space,
                            algo=tpe.suggest, # tpe = Tree-Structured Parzen Estimator
                            max_evals=TRIAL_COUNT,
                            trials=trials)

[HPO] metrics with {'lambda_l1': 0.017374955863332153, 'lambda_l2': 3.6749066668538037, 'learning_rate': 0.02567790850932432, 'max_depth': 36, 'min_data_in_leaf': 2, 'num_leaves': 1969} : acc= 58.33%
[HPO] metrics with {'lambda_l1': 0.28272711350148716, 'lambda_l2': 3.740656150509951, 'learning_rate': 0.9771002245814376, 'max_depth': 36, 'min_data_in_leaf': 88, 'num_leaves': 2198} : acc= 64.17%
[HPO] metrics with {'lambda_l1': 0.3327672217424602, 'lambda_l2': 1.235292345817427, 'learning_rate': 0.007154759400669562, 'max_depth': 86, 'min_data_in_leaf': 19, 'num_leaves': 3717} : acc= 47.92%
[HPO] metrics with {'lambda_l1': 0.8093069026098728, 'lambda_l2': 4.4907279520598244, 'learning_rate': 0.0019122450896481295, 'max_depth': 18, 'min_data_in_leaf': 88, 'num_leaves': 2703} : acc= 47.92%
[HPO] metrics with {'lambda_l1': 0.17013795018158295, 'lambda_l2': 1.1645869106231965, 'learning_rate': 0.016377417865204046, 'max_depth': 11, 'min_data_in_leaf': 93, 'num_leaves': 2837} : acc= 47.92%
[

In [467]:
# 각 학습된 모델을 이용하여 New dataset 에 대한 성능 도출

new_dataset_accuracy = []

for i, model in enumerate(models):
    preds_new = model.predict(X_test_new)
    accuracy_new = accuracy_score(preds_new, y_test_new)
    acc = f'{(100 * accuracy_new):6.2f}'

    new_dataset_accuracy.append(100 * accuracy_new)
    print(f'[NEW data] metrics for {i + 1}-th model : acc={acc}%')

[NEW data] metrics for 1-th model : acc= 58.67%
[NEW data] metrics for 2-th model : acc= 66.33%
[NEW data] metrics for 3-th model : acc= 47.67%
[NEW data] metrics for 4-th model : acc= 47.67%
[NEW data] metrics for 5-th model : acc= 47.67%
[NEW data] metrics for 6-th model : acc= 66.00%
[NEW data] metrics for 7-th model : acc= 65.33%
[NEW data] metrics for 8-th model : acc= 69.33%
[NEW data] metrics for 9-th model : acc= 60.67%
[NEW data] metrics for 10-th model : acc= 66.33%
[NEW data] metrics for 11-th model : acc= 67.33%
[NEW data] metrics for 12-th model : acc= 66.67%
[NEW data] metrics for 13-th model : acc= 51.33%
[NEW data] metrics for 14-th model : acc= 67.67%
[NEW data] metrics for 15-th model : acc= 47.67%
[NEW data] metrics for 16-th model : acc= 58.33%
[NEW data] metrics for 17-th model : acc= 47.67%
[NEW data] metrics for 18-th model : acc= 62.00%
[NEW data] metrics for 19-th model : acc= 67.67%
[NEW data] metrics for 20-th model : acc= 64.33%
[NEW data] metrics for 21-th 

In [468]:
# plotly 를 이용하여 성능 비교

WINDOW_SIZE = 30

import plotly.graph_objects as go

trial_nos = np.array(range(TRIAL_COUNT))
hpo_losses = np.array([trial['result']['loss'] for trial in trials.trials])
hpo_accuracy = (-1) * 100.0 * hpo_losses

# 성능 추이 표시
fig = go.Figure()
fig.add_trace(go.Scatter(x=trial_nos,
                         y=hpo_accuracy,
                         mode='markers',
                         marker={'size': 4, 'color': '#F70'},
                         name='HPO test dataset accuracy'))

fig.add_trace(go.Scatter(x=trial_nos,
                         y=new_dataset_accuracy,
                         mode='markers',
                         marker={'size': 4, 'color': '#07F'},
                         name='NEW test dataset accuracy'))

# 성능 추이의 이동평균 표시
hpo_accuracy_ma = np.convolve(hpo_accuracy,
                              np.ones(WINDOW_SIZE) / WINDOW_SIZE,
                              mode='valid')

new_dataset_accuracy_ma = np.convolve(new_dataset_accuracy,
                                      np.ones(WINDOW_SIZE) / WINDOW_SIZE,
                                      mode='valid')

fig.add_trace(go.Scatter(x=trial_nos[WINDOW_SIZE:],
                         y=hpo_accuracy_ma,
                         mode='lines',
                         line={'color': '#FA0'},
                         name='HPO test dataset accuracy (Moving Average)'))

fig.add_trace(go.Scatter(x=trial_nos[WINDOW_SIZE:],
                         y=new_dataset_accuracy_ma,
                         mode='lines',
                         line={'color': '#0AF'},
                         name='NEW test dataset accuracy (Moving Average)'))

# 차트 표시
fig.update_layout(width=1200,
                  height=600,
                  xaxis_title='trial No.',
                  yaxis_title='Accuracy (%)')

fig.show()

In [469]:
# 성능 차이 값의 변화

# 성능 차이 및 그 이동 평균
difference = [float(acc_hpo) - float(acc_new) for acc_hpo, acc_new in zip(hpo_accuracy, new_dataset_accuracy)]
difference_ma = np.convolve(difference, np.ones(WINDOW_SIZE) / WINDOW_SIZE, mode='valid')

# 그래프로 표시
fig = go.Figure()
fig.add_trace(go.Scatter(x=trial_nos,
                         y=difference,
                         mode='markers',
                         marker={'size': 5},
                         name='Accuracy Difference'))

fig.add_trace(go.Scatter(x=trial_nos[WINDOW_SIZE:],
                         y=difference_ma,
                         mode='lines',
                         name='Accuracy Difference (Moving Average)'))

fig.update_layout(width=1200,
                  height=600,
                  title='Difference : (HPO dataset ACC) - (NEW dataset ACC)',
                  xaxis_title='trial No.',
                  yaxis_title='Accuracy Difference (%)')

fig.show()

**3. Optuna 를 이용한 성능 테스트**

In [470]:
# Objective Function 정의

models = []

def objective_optuna(trial):
    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 1.0),
        'max_depth': trial.suggest_int('max_depth', 1, 100),
        'num_leaves': trial.suggest_int('num_leaves', 50, 4000),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 100),
        'lambda_l1': trial.suggest_uniform('lambda_l1', 0.0, 1.0),
        'lambda_l2': trial.suggest_uniform('lambda_l2', 0.0, 5.0)
    }

    model = lgb.LGBMClassifier(
        objective='multiclass',
        metric='multi_logloss',
        boosting_type='gbdt',
        num_iterations=10,  # 일반적으로 너무 작은 값이나, 여기서는 빠른 실험을 위해 사용
        verbosity=-1,
        **params
    )

    # LigthGBM 용 valid set 인 X_valid_lgbm, y_valid_lgbm 로 데이터를 분리
    X_train__, X_valid_lgbm, y_train__, y_valid_lgbm =\
        train_test_split(X_train_, y_train_,
                         test_size=0.25,
                         random_state=2025)

    # X_valid_lgbm, y_valid_lgbm 를 이용하여 LightGBM 모델을 valid
    model.fit(X_train__, y_train__,
              eval_set=[(X_valid_lgbm, y_valid_lgbm)])

    # HPO 용 test set인 X_test_hpo, y_test_hpo 를 이용하여 하이퍼파라미터 조합 성능 테스트
    preds_hpo = model.predict(X_test_hpo)

    accuracy_hpo = accuracy_score(preds_hpo, y_test_hpo)
    acc = f'{(100 * accuracy_hpo):6.2f}'

    print(f'[HPO] metrics with {params} : acc={acc}%')

    # 학습한 모델을 최종 테스트를 위해 배열에 저장
    models.append(model)

    # Accuracy 최대화
    return accuracy_hpo

In [471]:
# Optuna warning 미 출력 설정

import logging
optuna.logging.set_verbosity(logging.WARNING)

In [472]:
# 하이퍼파라미터 최적화 실시

study = optuna.create_study(direction="maximize")
study.optimize(objective_optuna, n_trials=TRIAL_COUNT)

[HPO] metrics with {'learning_rate': 0.011961135263093722, 'max_depth': 20, 'num_leaves': 3525, 'min_data_in_leaf': 22, 'lambda_l1': 0.3979650269067234, 'lambda_l2': 3.7851226111179495} : acc= 47.92%
[HPO] metrics with {'learning_rate': 0.17910195222766065, 'max_depth': 87, 'num_leaves': 944, 'min_data_in_leaf': 96, 'lambda_l1': 0.6791906422104688, 'lambda_l2': 4.565109924455812} : acc= 65.00%
[HPO] metrics with {'learning_rate': 0.38550086676403617, 'max_depth': 4, 'num_leaves': 2137, 'min_data_in_leaf': 55, 'lambda_l1': 0.18040491977378093, 'lambda_l2': 0.7209764830666199} : acc= 68.75%
[HPO] metrics with {'learning_rate': 0.012589723889484697, 'max_depth': 76, 'num_leaves': 754, 'min_data_in_leaf': 8, 'lambda_l1': 0.5879934026555943, 'lambda_l2': 1.9441415307792487} : acc= 47.92%
[HPO] metrics with {'learning_rate': 0.04316586881262808, 'max_depth': 75, 'num_leaves': 304, 'min_data_in_leaf': 66, 'lambda_l1': 0.5392474653456676, 'lambda_l2': 2.6488817515986} : acc= 60.00%
[HPO] metri

In [473]:
# 최적의 하이퍼파라미터 확인

best_params_optuna = study.best_params
print(f'Best Parameters : {best_params_optuna}')

Best Parameters : {'learning_rate': 0.4759536385740368, 'max_depth': 66, 'num_leaves': 872, 'min_data_in_leaf': 34, 'lambda_l1': 0.11542781261424431, 'lambda_l2': 0.2527873058857757}


In [474]:
# 각 학습된 모델을 이용하여 New dataset 에 대한 성능 도출

new_dataset_accuracy = []

for i, model in enumerate(models):
    preds_new = model.predict(X_test_new)
    accuracy_new = accuracy_score(preds_new, y_test_new)
    acc = f'{(100 * accuracy_new):6.2f}'

    new_dataset_accuracy.append(100 * accuracy_new)
    print(f'[NEW data] metrics for {i + 1}-th model : acc={acc}%')

[NEW data] metrics for 1-th model : acc= 47.67%
[NEW data] metrics for 2-th model : acc= 65.33%
[NEW data] metrics for 3-th model : acc= 67.00%
[NEW data] metrics for 4-th model : acc= 47.67%
[NEW data] metrics for 5-th model : acc= 59.33%
[NEW data] metrics for 6-th model : acc= 47.67%
[NEW data] metrics for 7-th model : acc= 65.00%
[NEW data] metrics for 8-th model : acc= 47.67%
[NEW data] metrics for 9-th model : acc= 47.67%
[NEW data] metrics for 10-th model : acc= 65.67%
[NEW data] metrics for 11-th model : acc= 62.33%
[NEW data] metrics for 12-th model : acc= 69.00%
[NEW data] metrics for 13-th model : acc= 64.67%
[NEW data] metrics for 14-th model : acc= 61.00%
[NEW data] metrics for 15-th model : acc= 68.67%
[NEW data] metrics for 16-th model : acc= 67.67%
[NEW data] metrics for 17-th model : acc= 64.67%
[NEW data] metrics for 18-th model : acc= 54.00%
[NEW data] metrics for 19-th model : acc= 67.67%
[NEW data] metrics for 20-th model : acc= 66.00%
[NEW data] metrics for 21-th 

In [475]:
# plotly 를 이용하여 성능 비교

WINDOW_SIZE = 30

import plotly.graph_objects as go

trial_nos = np.array(range(TRIAL_COUNT))
trials_df = study.trials_dataframe()
hpo_accuracy = 100 * trials_df['value'].to_numpy()

# 성능 추이 표시
fig = go.Figure()
fig.add_trace(go.Scatter(x=trial_nos,
                         y=hpo_accuracy,
                         mode='markers',
                         marker={'size': 4, 'color': '#F70'},
                         name='HPO test dataset accuracy'))

fig.add_trace(go.Scatter(x=trial_nos,
                         y=new_dataset_accuracy,
                         mode='markers',
                         marker={'size': 4, 'color': '#07F'},
                         name='NEW test dataset accuracy'))

# 성능 추이의 이동평균 표시
hpo_accuracy_ma = np.convolve(hpo_accuracy,
                              np.ones(WINDOW_SIZE) / WINDOW_SIZE,
                              mode='valid')

new_dataset_accuracy_ma = np.convolve(new_dataset_accuracy,
                                      np.ones(WINDOW_SIZE) / WINDOW_SIZE,
                                      mode='valid')

fig.add_trace(go.Scatter(x=trial_nos[WINDOW_SIZE:],
                         y=hpo_accuracy_ma,
                         mode='lines',
                         line={'color': '#FA0'},
                         name='HPO test dataset accuracy (Moving Average)'))

fig.add_trace(go.Scatter(x=trial_nos[WINDOW_SIZE:],
                         y=new_dataset_accuracy_ma,
                         mode='lines',
                         line={'color': '#0AF'},
                         name='NEW test dataset accuracy (Moving Average)'))

# 차트 표시
fig.update_layout(width=1200,
                  height=600,
                  xaxis_title='trial No.',
                  yaxis_title='Accuracy (%)')

fig.show()

In [476]:
# 성능 차이 값의 변화

# 성능 차이 및 그 이동 평균
difference = [float(acc_hpo) - float(acc_new) for acc_hpo, acc_new in zip(hpo_accuracy, new_dataset_accuracy)]
difference_ma = np.convolve(difference, np.ones(WINDOW_SIZE) / WINDOW_SIZE, mode='valid')

# 그래프로 표시
fig = go.Figure()
fig.add_trace(go.Scatter(x=trial_nos,
                         y=difference,
                         mode='markers',
                         marker={'size': 5},
                         name='Accuracy Difference'))

fig.add_trace(go.Scatter(x=trial_nos[WINDOW_SIZE:],
                         y=difference_ma,
                         mode='lines',
                         name='Accuracy Difference (Moving Average)'))

fig.update_layout(width=1200,
                  height=600,
                  title='Difference : (HPO dataset ACC) - (NEW dataset ACC)',
                  xaxis_title='trial No.',
                  yaxis_title='Accuracy Difference (%)')

fig.show()